In [1]:
!pip install fire

  Stored in directory: /root/.cache/pip/wheels/2a/1a/4d/6b30377c3051e76559d1185c1dbbfff15aed31f87acdd14c22
Successfully built fire


In [2]:
!git clone https://github.com/dhlee347/pytorchic-bert.git

Cloning into 'pytorchic-bert'...
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 169 (delta 1), reused 0 (delta 0), pack-reused 162
Receiving objects: 100% (169/169), 55.42 KiB | 2.41 MiB/s, done.
Resolving deltas: 100% (94/94), done.


In [0]:
!cp pytorchic-bert/*.py .
!cp -r pytorchic-bert/config .

In [4]:
!wget https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip

--2019-06-28 12:17:59--  https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.214.128, 2607:f8b0:4001:c03::80
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.214.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 407727028 (389M) [application/zip]
Saving to: ‘uncased_L-12_H-768_A-12.zip’

uncased_L-12_H-768_ 100%[===================>] 388.84M   100MB/s    in 3.9s    

2019-06-28 12:18:03 (100 MB/s) - ‘uncased_L-12_H-768_A-12.zip’ saved [407727028/407727028]



In [5]:
!unzip uncased_L-12_H-768_A-12.zip

Archive:  uncased_L-12_H-768_A-12.zip
   creating: uncased_L-12_H-768_A-12/
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.meta  
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001  
  inflating: uncased_L-12_H-768_A-12/vocab.txt  
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.index  
  inflating: uncased_L-12_H-768_A-12/bert_config.json  


In [6]:
!mkdir mrpc
!wget -O mrpc/train.tsv https://docs.google.com/uc?id='1m5ft6FSPB5ORy6CMu7612PK3gBD81Hm0'
!wget -O mrpc/dev.tsv https://docs.google.com/uc?id='1oex3jntQ6LIfNi4LacKqBF15G71g2KeB'

--2019-06-28 12:20:24--  https://docs.google.com/uc?id=1m5ft6FSPB5ORy6CMu7612PK3gBD81Hm0
Resolving docs.google.com (docs.google.com)... 108.177.111.113, 108.177.111.101, 108.177.111.139, ...
Connecting to docs.google.com (docs.google.com)|108.177.111.113|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0s-6k-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/ql139oo12lri1fqblgtv4bcub9tvid6u/1561723200000/06007603302574522241/*/1m5ft6FSPB5ORy6CMu7612PK3gBD81Hm0 [following]
--2019-06-28 12:20:24--  https://doc-0s-6k-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/ql139oo12lri1fqblgtv4bcub9tvid6u/1561723200000/06007603302574522241/*/1m5ft6FSPB5ORy6CMu7612PK3gBD81Hm0
Resolving doc-0s-6k-docs.googleusercontent.com (doc-0s-6k-docs.googleusercontent.com)... 209.85.200.132, 2607:f8b0:4001:c16::84
Connecting to doc-0s-6k-docs.googleusercontent.com (doc-0s-6k-docs.googleusercontent.com)|209.85.

In [7]:
!head mrpc/train.tsv

﻿Quality	#1 ID	#2 ID	#1 String	#2 String
1	702876	702977	Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .	Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .
0	2108705	2108831	Yucaipa owned Dominick 's before selling the chain to Safeway in 1998 for $ 2.5 billion .	Yucaipa bought Dominick 's in 1995 for $ 693 million and sold it to Safeway for $ 1.8 billion in 1998 .
1	1330381	1330521	They had published an advertisement on the Internet on June 10 , offering the cargo for sale , he added .	On June 10 , the ship 's owners had published an advertisement on the Internet , offering the explosives for sale .
0	3344667	3344648	Around 0335 GMT , Tab shares were up 19 cents , or 4.4 % , at A $ 4.56 , having earlier set a record high of A $ 4.57 .	Tab shares jumped 20 cents , or 4.6 % , to set a record closing high at A $ 4.57 .
1	1236820	1236712	The stock rose $ 2.11 , or about 11 

In [9]:
!ls

checkpoint.py  optim.py		train.py
classify.py    pretrain.py	uncased_L-12_H-768_A-12
config	       pytorchic-bert	uncased_L-12_H-768_A-12.zip
models.py      sample_data	utils.py
mrpc	       tokenization.py


In [0]:
import csv, fire, itertools

import torch
import torch.nn as nn

import train, models, optim, tokenization, classify
from utils import *
from torch.utils.data import Dataset, DataLoader

In [11]:
cfg = train.Config.from_json('config/train_mrpc.json')
model_cfg = models.Config.from_json('config/bert_base.json')
print(cfg)
print(model_cfg)

Config(seed=42, batch_size=32, lr=2e-05, n_epochs=3, warmup=0.1, save_steps=100, total_steps=345)
Config(vocab_size=30522, dim=768, n_layers=12, n_heads=12, dim_ff=3072, p_drop_hidden=0.1, p_drop_attn=0.1, max_len=512, n_segments=2)


In [0]:
max_len=128
set_seeds(42)
tokenizer = tokenization.FullTokenizer(vocab_file='uncased_L-12_H-768_A-12/vocab.txt', do_lower_case=True)

In [13]:
!tail -n 1000 uncased_L-12_H-768_A-12/vocab.txt | head -n 10

##ssee
colossal
foreigner
vet
freaks
patrice
rosewood
triassic
upstate
##pkins


In [0]:
pipeline = [classify.Tokenizing(tokenizer.convert_to_unicode, tokenizer.tokenize),
                classify.AddSpecialTokensWithTruncation(max_len),
                classify.TokenIndexing(tokenizer.convert_tokens_to_ids,
                classify.MRPC.labels, max_len)]

In [0]:
def get_instances(lines):
    for line in itertools.islice(lines, 1, 3): # skip header
        yield line[0], line[3], line[4] # label, text_a, text_b

def get_dataset(file, pipeline):
  data = []
  with open('mrpc/dev.tsv', "r") as f:
      # list of splitted lines : line is also list
      lines = csv.reader(f, delimiter='\t', quotechar=None)
      for instance in get_instances(lines): # instance : tuple of fields
          for proc in pipeline: # a bunch of pre-processing
              instance = proc(instance)
          data.append(instance)
  return data

In [20]:
  print(f"\n* Take a look at the dataset according to pipeline (max_len : {max_len}):\n")
  for i in range(len(pipeline)+1):
      print("Preprocessing Pipeline : ", end="")
      for proc in pipeline[:i]:
          print(type(proc).__name__, end=", ")
      dataset =get_dataset('mrpc/dev.tsv',pipeline[:i])
      print('\n', dataset[0], '\n', dataset[1], '\n')



* Take a look at the dataset according to pipeline (max_len : 128):

Preprocessing Pipeline : 
 ('1', "He said the foodservice pie business doesn 't fit the company 's long-term growth strategy .", '" The foodservice pie business does not fit our long-term growth strategy .') 
 ('0', 'Magnarelli said Racicot hated the Iraqi regime and looked forward to using his long years of training in the war .', 'His wife said he was " 100 percent behind George Bush " and looked forward to using his years of training in the war .') 

Preprocessing Pipeline : Tokenizing, 
 ('1', ['he', 'said', 'the', 'foods', '##er', '##vic', '##e', 'pie', 'business', 'doesn', "'", 't', 'fit', 'the', 'company', "'", 's', 'long', '-', 'term', 'growth', 'strategy', '.'], ['"', 'the', 'foods', '##er', '##vic', '##e', 'pie', 'business', 'does', 'not', 'fit', 'our', 'long', '-', 'term', 'growth', 'strategy', '.']) 
 ('0', ['magna', '##relli', 'said', 'ra', '##cic', '##ot', 'hated', 'the', 'iraqi', 'regime', 'and', 'look

In [0]:
dataset = classify.MRPC('mrpc/train.tsv', pipeline)
data_iter = DataLoader(dataset, batch_size=cfg.batch_size, shuffle=True)

model = classify.Classifier(model_cfg, len(classify.MRPC.labels))
criterion = nn.CrossEntropyLoss()

In [24]:
save_dir = '.'
trainer = train.Trainer(cfg,
                        model,
                        data_iter,
                        optim.optim4GPU(cfg, model),
                        save_dir, get_device())

cuda (1 GPUs)


In [26]:
def get_loss(model, batch, global_step): # make sure loss is a scalar tensor
    input_ids, segment_ids, input_mask, label_id = batch
    logits = model(input_ids, segment_ids, input_mask)
    loss = criterion(logits, label_id)
    return loss

trainer.train(get_loss, model_file=None, pretrain_file='uncased_L-12_H-768_A-12/bert_model.ckpt', data_parallel=False)

Loading the pretrained model from uncased_L-12_H-768_A-12/bert_model.ckpt


Iter (loss=X.XXX):   0%|          | 0/115 [00:00<?, ?it/s]

Epoch 1/3 : Average Loss 0.556


Iter (loss=X.XXX):   0%|          | 0/115 [00:00<?, ?it/s]

Epoch 2/3 : Average Loss 0.273


Iter (loss=0.062): 100%|██████████| 115/115 [01:45<00:00,  1.22it/s]


Epoch 3/3 : Average Loss 0.089


In [0]:
dataset = classify.MRPC('mrpc/dev.tsv', pipeline)
data_iter = DataLoader(dataset, batch_size=cfg.batch_size, shuffle=True)

In [28]:
save_dir = '.'
trainer = train.Trainer(cfg,
                        model,
                        data_iter,
                        optim.optim4GPU(cfg, model),
                        save_dir, get_device())

cuda (1 GPUs)


In [29]:
def evaluate(model, batch):
    input_ids, segment_ids, input_mask, label_id = batch
    logits = model(input_ids, segment_ids, input_mask)
    _, label_pred = logits.max(1)
    result = (label_pred == label_id).float() #.cpu().numpy()
    accuracy = result.mean()
    return accuracy, result

results = trainer.eval(evaluate, model_file='model_steps_345.pt', data_parallel=False)


Loading the model from model_steps_345.pt


Iter(acc=0.875): 100%|██████████| 13/13 [00:04<00:00,  3.45it/s]


In [30]:
total_accuracy = torch.cat(results).mean().item()
print('Accuracy:', total_accuracy)

Accuracy: 0.8406863212585449
